In [1]:
import os
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG

import tensorflow
from keras.models import load_model
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

colab = False  # Type True if using Google Colab, type False if using Jupyter Notebook
if colab:
    from google.colab import drive
    from zipfile import ZipFile

tensorflow.random.set_seed(42)

In [85]:
model_es = load_model('./model_data_softmax/')

In [3]:
img_width, img_height = 224, 224 
channels = 3
batch_size = 64
#num_images= 50
image_arr_size= img_width * img_height * channels

In [84]:
train_data_path = './Images/'
test_data_path = './Testing_images/'

In [5]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rotation_range= 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,   
    validation_split=0.2,

)

train_generator = train_datagen.flow_from_directory(  
    train_data_path,  
    target_size= (img_width, img_height), 
    color_mode= 'rgb',
    batch_size= batch_size,  
    class_mode= 'categorical',
    subset='training',
    shuffle= True, 
    seed= 1337
) 

Found 17230 images belonging to 120 classes.


### Make a dict with the class and their indeces 

In [6]:
train_class_indeces = train_generator.class_indices

In [7]:
train_class_ind = {}
for k, i in train_class_indeces.items():
    key = k.replace('_', ' ').title()
    train_class_ind[i] = key

In [8]:
train_class_ind

{0: 'Affenpinscher',
 1: 'Afghan',
 2: 'African Wild Dog',
 3: 'Airedale',
 4: 'American Staffordshire Terrier',
 5: 'Appenzeller',
 6: 'Australian Terrier',
 7: 'Basenji',
 8: 'Basset',
 9: 'Beagle',
 10: 'Bedlington Terrier',
 11: 'Bernese Mountain Dog',
 12: 'Black-And-Tan Coonhound',
 13: 'Blenheim Spaniel',
 14: 'Bloodhound',
 15: 'Bluetick',
 16: 'Border Collie',
 17: 'Border Terrier',
 18: 'Borzoi',
 19: 'Boston Bull',
 20: 'Bouvier Des Flandres',
 21: 'Boxer',
 22: 'Brabancon Griffon',
 23: 'Briard',
 24: 'Brittany Spaniel',
 25: 'Bull Mastiff',
 26: 'Cairn',
 27: 'Cardigan',
 28: 'Chesapeake Bay Retriever',
 29: 'Chihuahua',
 30: 'Chow',
 31: 'Clumber',
 32: 'Cocker Spaniel',
 33: 'Collie',
 34: 'Curly Coated Retriever',
 35: 'Dandie Dinmont',
 36: 'Dhole',
 37: 'Dingo',
 38: 'Doberman',
 39: 'English Foxhound',
 40: 'English Setter',
 41: 'English Springer',
 42: 'Entlebucher',
 43: 'Eskimo Dog',
 44: 'Flat Coated Retriever',
 45: 'French Bulldog',
 46: 'German Shepherd',
 47

In [9]:
train_class_ind_loc = {}
for k, i in train_class_ind.items():
    train_class_ind_loc[i] = k

In [10]:
train_class_ind_loc

{'Affenpinscher': 0,
 'Afghan': 1,
 'African Wild Dog': 2,
 'Airedale': 3,
 'American Staffordshire Terrier': 4,
 'Appenzeller': 5,
 'Australian Terrier': 6,
 'Basenji': 7,
 'Basset': 8,
 'Beagle': 9,
 'Bedlington Terrier': 10,
 'Bernese Mountain Dog': 11,
 'Black-And-Tan Coonhound': 12,
 'Blenheim Spaniel': 13,
 'Bloodhound': 14,
 'Bluetick': 15,
 'Border Collie': 16,
 'Border Terrier': 17,
 'Borzoi': 18,
 'Boston Bull': 19,
 'Bouvier Des Flandres': 20,
 'Boxer': 21,
 'Brabancon Griffon': 22,
 'Briard': 23,
 'Brittany Spaniel': 24,
 'Bull Mastiff': 25,
 'Cairn': 26,
 'Cardigan': 27,
 'Chesapeake Bay Retriever': 28,
 'Chihuahua': 29,
 'Chow': 30,
 'Clumber': 31,
 'Cocker Spaniel': 32,
 'Collie': 33,
 'Curly Coated Retriever': 34,
 'Dandie Dinmont': 35,
 'Dhole': 36,
 'Dingo': 37,
 'Doberman': 38,
 'English Foxhound': 39,
 'English Setter': 40,
 'English Springer': 41,
 'Entlebucher': 42,
 'Eskimo Dog': 43,
 'Flat Coated Retriever': 44,
 'French Bulldog': 45,
 'German Shepherd': 46,
 'G

### Predicting mixed chow-husky

In [11]:
test_datagen = ImageDataGenerator(
    rescale= 1./255, 
    validation_split=0.2,
)

In [12]:
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,  
    class_mode= 'categorical',
    shuffle=False
)

Found 9 images belonging to 1 classes.


In [13]:
file_path = test_generator.filenames

In [14]:
file_path

['chow_siberian husky/00.jpeg',
 'chow_siberian husky/001.jpeg',
 'chow_siberian husky/002.jpeg',
 'chow_siberian husky/003.jpeg',
 'chow_siberian husky/004.jpeg',
 'chow_siberian husky/005.jpeg',
 'chow_siberian husky/006.jpeg',
 'chow_siberian husky/008.jpeg',
 'chow_siberian husky/009.jpeg']

### Testing it on Model 1

In [15]:
preds = model_es.predict(test_generator)

In [16]:
preds.shape

(9, 120)

In [17]:
preds[8][99]

2.4995596e-05

In [18]:
preds

array([[4.2258598e-05, 4.0677699e-04, 3.9922583e-05, ..., 2.1967088e-04,
        2.0577078e-04, 2.5885087e-05],
       [4.0564964e-07, 8.9490123e-07, 7.4100655e-08, ..., 7.1439126e-07,
        2.5207992e-06, 2.1443860e-07],
       [1.7680847e-07, 2.2456284e-06, 8.1659630e-07, ..., 7.5932292e-07,
        3.6317770e-06, 8.9889011e-07],
       ...,
       [4.3432974e-05, 5.8837657e-05, 6.3393498e-05, ..., 6.4199201e-05,
        7.8271267e-05, 4.7045574e-05],
       [7.3148425e-05, 1.1252036e-04, 7.0438124e-05, ..., 1.9984017e-04,
        1.7119224e-04, 1.9169549e-04],
       [8.0515514e-05, 1.2822423e-05, 9.4722154e-06, ..., 7.4643244e-06,
        2.3884699e-05, 2.0231553e-05]], dtype=float32)

In [19]:
preds_sort = np.argsort(preds, axis=1)

In [20]:
preds_sort[0]

array([ 78, 107,  98,  10,  12,  35,  22, 119,  97, 102,  31,   2,   0,
       106,  42,  59,  57, 103,  19,   5,  50,  51, 114,  39,  17, 116,
         6,  41, 113, 112,  58,  45,  76,  28,  53,  40,  96, 100,  13,
        48,  24,  65,   8,  20,  38,   4,  11,  79,  80,  70,  73,  21,
        26,  74,  47,  86,  82,  55,  25, 111, 118,  90, 117, 109,  68,
        88,  89,  71,  83, 104,  34,   9,  61,  15,  64,  81,  91,   3,
         1,  92, 115,  23, 105,  32,  72, 101,  44,  94,  77,  67,  14,
        93,  56,  75,  84,  66,  18,  60, 110,  69, 108,  52,  27,  43,
        29,  62,  63,  16,   7,  95,  46,  36,  54,  49,  85,  99,  37,
        87,  30,  33])

In [21]:
preds_sort[0][-10:]

array([46, 36, 54, 49, 85, 99, 37, 87, 30, 33])

In [22]:
for arr in preds_sort:
    print(arr[-10:])

[46 36 54 49 85 99 37 87 30 33]
[ 62 101  69  33  49  87  94  43 108  30]
[ 43 108  94  54  79  69  36  62  87  30]
[ 71 108  85  49 101  43  87  94  33  30]
[63 16 33 46 49 43 37 87 30 85]
[119  87  79  80  71  46  29 101  43  85]
[ 69  37  36  85 108  46  33 101  71  43]
[ 33  54  49  69 108 101  71  94  43  30]
[ 33  48  43  94  20 108  77  95  54  30]


In [23]:
pred_dogs = []
i = 0
for arr in preds_sort:
    temp_list = [(train_class_ind[index], preds[i][index]) for index in arr[-10:]]
    pred_dogs.append(temp_list)
    i +=1

In [24]:
pred_dogs[2]

[('Eskimo Dog', 1.5929532e-05),
 ('Tibetan Mastiff', 2.3478155e-05),
 ('Samoyed', 3.238641e-05),
 ('Groenendael', 3.7079753e-05),
 ('Norwegian Elkhound', 6.967402e-05),
 ('Leonberg', 9.51926e-05),
 ('Dhole', 0.00050524954),
 ('Keeshond', 0.0015143652),
 ('Pomeranian', 0.0022830039),
 ('Chow', 0.99522096)]

In [25]:
len(test_generator.filenames)

9

In [26]:
predicted_breeds = {}
i = 0
for arr in preds_sort:
    temp_list = [train_class_ind[index] for index in arr[-10:]]
    predicted_breeds[file_path[i]] = temp_list
    i +=1

In [27]:
predicted_breeds

{'chow_siberian husky/00.jpeg': ['German Shepherd',
  'Dhole',
  'Groenendael',
  'Golden Retriever',
  'Pembroke Corgi',
  'Shetland Sheepdog',
  'Dingo',
  'Pomeranian',
  'Chow',
  'Collie'],
 'chow_siberian husky/001.jpeg': ['Keeshond',
  'Siberian Husky',
  'Leonberg',
  'Collie',
  'Golden Retriever',
  'Pomeranian',
  'Samoyed',
  'Eskimo Dog',
  'Tibetan Mastiff',
  'Chow'],
 'chow_siberian husky/002.jpeg': ['Eskimo Dog',
  'Tibetan Mastiff',
  'Samoyed',
  'Groenendael',
  'Norwegian Elkhound',
  'Leonberg',
  'Dhole',
  'Keeshond',
  'Pomeranian',
  'Chow'],
 'chow_siberian husky/003.jpeg': ['Malamute',
  'Tibetan Mastiff',
  'Pembroke Corgi',
  'Golden Retriever',
  'Siberian Husky',
  'Eskimo Dog',
  'Pomeranian',
  'Samoyed',
  'Collie',
  'Chow'],
 'chow_siberian husky/004.jpeg': ['Kelpie',
  'Border Collie',
  'Collie',
  'German Shepherd',
  'Golden Retriever',
  'Eskimo Dog',
  'Dingo',
  'Pomeranian',
  'Chow',
  'Pembroke Corgi'],
 'chow_siberian husky/005.jpeg': ['Y

In [28]:
pred_breeds = pd.DataFrame(data=predicted_breeds.items(), columns=['filepath', 'predicted_breeds'])

In [29]:
pred_breeds.head()

,filepath,predicted_breeds
0,chow_siberian husky/00.jpeg,"[German Shepherd, Dhole, Groenendael, Golden R..."
1,chow_siberian husky/001.jpeg,"[Keeshond, Siberian Husky, Leonberg, Collie, G..."
2,chow_siberian husky/002.jpeg,"[Eskimo Dog, Tibetan Mastiff, Samoyed, Groenen..."
3,chow_siberian husky/003.jpeg,"[Malamute, Tibetan Mastiff, Pembroke Corgi, Go..."
4,chow_siberian husky/004.jpeg,"[Kelpie, Border Collie, Collie, German Shepher..."


In [30]:
true_breeds = ['Chow', 'Siberian Husky']

for ix in pred_breeds.index:
    predicted_breeds = pred_breeds.loc[ix, 'predicted_breeds']
    print(any([breed in true_breeds for breed in predicted_breeds]))

True
True
True
True
True
True
True
True
True


In [31]:
type(pred_breeds.loc[0, 'predicted_breeds'])

list

In [32]:
pred_breeds.to_csv('predicted_breeds', index=False)

#### Making df with true values for mixed breeds

In [33]:
file_path

['chow_siberian husky/00.jpeg',
 'chow_siberian husky/001.jpeg',
 'chow_siberian husky/002.jpeg',
 'chow_siberian husky/003.jpeg',
 'chow_siberian husky/004.jpeg',
 'chow_siberian husky/005.jpeg',
 'chow_siberian husky/006.jpeg',
 'chow_siberian husky/008.jpeg',
 'chow_siberian husky/009.jpeg']

In [34]:
list_breeds = []
for i in file_path:
    i = i[:i.find('/')]
    list_breeds.append(i.split('_'))

In [35]:
list_breeds

[['chow', 'siberian husky'],
 ['chow', 'siberian husky'],
 ['chow', 'siberian husky'],
 ['chow', 'siberian husky'],
 ['chow', 'siberian husky'],
 ['chow', 'siberian husky'],
 ['chow', 'siberian husky'],
 ['chow', 'siberian husky'],
 ['chow', 'siberian husky']]

In [36]:
y_true = np.zeros((len(file_path), 120))

In [37]:
y_true.shape

(9, 120)

In [38]:
y_true

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
breed = []
j = 0
for i in list_breeds:
    for k, v in train_class_ind_loc.items():
        if k.lower() in i:
            y_true[j][v] = 1
    j += 1

In [40]:
y_true[8]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

### Testing mixed breeds with 'sigmoid_model'

In [68]:
#model_2 = load_model('./model_data_2')

In [69]:
preds_2 = model_2.predict(test_generator)

In [334]:
preds_2[0]

array([3.3986568e-04, 1.6006827e-04, 6.0280370e-05, 1.1988282e-03,
       2.4849533e-05, 3.7597576e-05, 3.0726194e-04, 8.0754253e-05,
       2.3773313e-04, 3.2940507e-04, 8.4830615e-05, 4.3936448e-05,
       2.4540197e-05, 4.6412424e-05, 7.0904527e-05, 1.5139580e-04,
       3.2006785e-05, 1.1192752e-04, 3.1882524e-04, 3.7323673e-05,
       3.1498075e-04, 1.6632676e-04, 1.1498617e-04, 1.9833446e-04,
       1.7348317e-05, 4.4947863e-04, 1.8702252e-05, 4.2396965e-05,
       8.1256861e-05, 1.4123321e-04, 1.1603903e-05, 6.9513048e-06,
       8.0176884e-05, 6.0826431e-05, 1.0951416e-05, 4.2911295e-05,
       2.4101100e-05, 1.1159998e-04, 1.5822053e-04, 1.3894558e-05,
       6.7568137e-05, 1.3814633e-05, 2.4125660e-05, 9.7612292e-06,
       6.4284985e-05, 1.9494895e-05, 6.8881025e-05, 1.4375609e-05,
       6.8619811e-05, 2.0247006e-05, 7.1917907e-06, 6.5665954e-05,
       4.2967436e-05, 3.2160722e-05, 3.0750036e-04, 1.0877456e-04,
       2.6685625e-02, 7.9809935e-05, 9.0331436e-05, 3.3586261e

In [357]:
preds_encoded = np.where(preds_2 >= .05, 1, 0)

In [358]:
preds_encoded[4]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0])

### See how it does with only one breed - unseen pics

In [54]:
non_mixed_path = './Testing_images/'

In [55]:
non_mixed_test_datagen = ImageDataGenerator(
    rescale= 1./255, 
)

In [56]:
non_mixed_test_generator = non_mixed_test_datagen.flow_from_directory(
    non_mixed_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,  
    class_mode= 'categorical',
    shuffle=False
)

Found 3633 images belonging to 120 classes.


In [57]:
label_names = list(non_mixed_test_generator.class_indices.keys())

In [66]:
train_class_indeces

{'Affenpinscher': 0,
 'Afghan': 1,
 'African Wild Dog': 2,
 'Airedale': 3,
 'American Staffordshire Terrier': 4,
 'Appenzeller': 5,
 'Australian Terrier': 6,
 'Basenji': 7,
 'Basset': 8,
 'Beagle': 9,
 'Bedlington Terrier': 10,
 'Bernese Mountain Dog': 11,
 'Black-and-Tan Coonhound': 12,
 'Blenheim Spaniel': 13,
 'Bloodhound': 14,
 'Bluetick': 15,
 'Border Collie': 16,
 'Border Terrier': 17,
 'Borzoi': 18,
 'Boston Bull': 19,
 'Bouvier Des Flandres': 20,
 'Boxer': 21,
 'Brabancon Griffon': 22,
 'Briard': 23,
 'Brittany Spaniel': 24,
 'Bull Mastiff': 25,
 'Cairn': 26,
 'Cardigan': 27,
 'Chesapeake Bay Retriever': 28,
 'Chihuahua': 29,
 'Chow': 30,
 'Clumber': 31,
 'Cocker Spaniel': 32,
 'Collie': 33,
 'Curly Coated Retriever': 34,
 'Dandie Dinmont': 35,
 'Dhole': 36,
 'Dingo': 37,
 'Doberman': 38,
 'English Foxhound': 39,
 'English Setter': 40,
 'English Springer': 41,
 'Entlebucher': 42,
 'Eskimo Dog': 43,
 'Flat Coated Retriever': 44,
 'French Bulldog': 45,
 'German Shepherd': 46,
 'G

In [72]:
test_class_indeces = non_mixed_test_generator.class_indices
(len(train_class_indeces)

SyntaxError: unexpected EOF while parsing (3582114756.py, line 2)

In [83]:
j = 0
for i in range(len(train_class_indeces)):
    if list(train_class_indeces.keys())[i] == list(test_class_indeces.keys())[i]:
        j +=1
print(j)

120


In [58]:
#non_mixed_test_generator.classes

In [59]:
#non_mixed_test_generator.filepaths

In [60]:
file_paths = non_mixed_test_generator.filenames

In [61]:
file_paths[0]

'Affenpinscher/001.jpg'

In [62]:
non_mixed_preds = model_es.predict(non_mixed_test_generator)


KeyboardInterrupt



In [50]:
single_breed_preds = np.argmax(non_mixed_preds, axis=1)

In [52]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [53]:
print(classification_report(non_mixed_test_generator.classes, single_breed_preds, target_names=label_names))

                                precision    recall  f1-score   support

                 Affenpinscher       0.76      1.00      0.87        13
                        Afghan       0.98      0.98      0.98        50
              African Wild Dog       1.00      0.98      0.99        50
                      Airedale       0.92      0.90      0.91        50
American Staffordshire Terrier       0.77      0.88      0.82        50
                   Appenzeller       1.00      0.25      0.40        12
            Australian Terrier       0.50      0.92      0.65        12
                       Basenji       1.00      1.00      1.00        50
                        Basset       1.00      0.94      0.97        50
                        Beagle       0.92      0.96      0.94        50
            Bedlington Terrier       1.00      1.00      1.00        13
          Bernese Mountain Dog       1.00      0.96      0.98        50
       Black-and-Tan Coonhound       0.67      0.91      0.77  

#### Try the newest model

In [ ]:
def unpack_predicts(preds, file_path):
    preds_sort = np.argsort(preds, axis=1)
    predicted_breeds = {}
    i = 0
    for arr in preds_sort:
        temp_list = [train_class_ind[index] for index in arr[-10:]]
        predicted_breeds[file_path[i]] = temp_list
        i +=1
    pred_breeds = pd.DataFrame(data=predicted_breeds.items(), columns=['filepath', 'predicted_breeds'])
    
    return pred_breeds

In [ ]:
non_mixed_predictions = unpack_predicts(non_mixed_preds, non_mixed_test_generator.filepaths)

In [ ]:
non_mixed_predictions 

,filepath,predicted_breeds
0,./non_mixed_data/Affenpinscher/001.jpg,"[German Shepherd, Schipperke, Groenendael, American Staffordshire Terrier, Malinois, Silky Terrier, Staffordshire Bullterrier, Airedale, Toy Poodle, Shih-Tzu]"
1,./non_mixed_data/Affenpinscher/002.jpg,"[Miniature Poodle, Otterhound, German Shepherd, Airedale, Eskimo Dog, Golden Retriever, Greater Swiss Mountain Dog, American Staffordshire Terrier, Toy Poodle, Shih-Tzu]"
2,./non_mixed_data/Affenpinscher/003.jpg,"[Curly Coated Retriever, Silky Terrier, Samoyed, Otterhound, French Bulldog, German Short Haired Pointer, Airedale, American Staffordshire Terrier, Toy Poodle, Shih-Tzu]"
3,./non_mixed_data/Affenpinscher/004.jpg,"[Rottweiler, Groenendael, Miniature Poodle, Greater Swiss Mountain Dog, Dhole, American Staffordshire Terrier, German Short Haired Pointer, French Bulldog, Toy Poodle, Shih-Tzu]"
4,./non_mixed_data/Affenpinscher/005.jpg,"[Samoyed, Malinois, Silky Terrier, Chesapeake Bay Retriever, German Shepherd, American Staffordshire Terrier, Toy Poodle, German Short Haired Pointer, French Bulldog, Shih-Tzu]"
...,...,...
3628,./non_mixed_data/Yorkshire Terrier/46.jpg,"[English Springer, Shih-Tzu, Brittany Spaniel, Golden Retriever, Curly Coated Retriever, Dandie Dinmont, Miniature Poodle, Gordon Setter, Dhole, Entlebucher]"
3629,./non_mixed_data/Yorkshire Terrier/47.jpg,"[American Staffordshire Terrier, Shih-Tzu, Toy Poodle, English Springer, Dandie Dinmont, Curly Coated Retriever, Basenji, Gordon Setter, Entlebucher, Dhole]"
3630,./non_mixed_data/Yorkshire Terrier/48.jpg,"[Miniature Poodle, Eskimo Dog, Curly Coated Retriever, Basenji, American Staffordshire Terrier, Golden Retriever, French Bulldog, Entlebucher, Gordon Setter, Dhole]"
3631,./non_mixed_data/Yorkshire Terrier/49.jpg,"[Eskimo Dog, Basenji, Doberman, Shih-Tzu, Curly Coated Retriever, Dandie Dinmont, Gordon Setter, Dhole, Entlebucher, English Springer]"


In [ ]:
#pd.set_option('display.max_colwidth', None)

In [ ]:
non_mixed_predictions['predicted_breeds'][0][-1]

'Airedale'

In [ ]:
def get_dominant_pred(row):
    dom_pred = row['predicted_breeds'][-1]
    
    return dom_pred

In [ ]:
non_mixed_predictions['pred_1'] = non_mixed_predictions.apply(get_dominant_pred, axis=1)

In [ ]:
non_mixed_predictions

,filepath,predicted_breeds,pred_1
0,Airedale/001.jpeg,"[Australian Terrier, German Shepherd, Border Terrier, Standard Poodle, Irish Terrier, Lakeland Terrier, Giant Schnauzer, Otterhound, Wire-Haired Fox Terrier, Airedale]",Airedale
1,Chesapeake Bay Retriever/001.jpeg,"[American Staffordshire Terrier, Golden Retriever, Irish Wolfhound, Irish Setter, German Short-Haired Pointer, Flat-Coated Retriever, Labrador Retriever, Standard Poodle, Curly-Coated Retriever, Chesapeake Bay Retriever]",Chesapeake Bay Retriever
2,Chihuahua/001.jpg,"[Appenzeller, Saluki, Ibizan Hound, Brabancon Griffon, Dingo, Basenji, Kelpie, Miniature Pinscher, Chihuahua, Toy Terrier]",Toy Terrier
3,Golden Retriever/001.jpeg,"[Saluki, Clumber, Flat-Coated Retriever, Tibetan Mastiff, Kuvasz, Irish Setter, English Setter, Labrador Retriever, Cocker Spaniel, Golden Retriever]",Golden Retriever
4,Maltese/001.jpeg,"[Sealyham Terrier, Great Pyrenees, Silky Terrier, West Highland White Terrier, Dandie Dinmont, Tibetan Mastiff, Shih-Tzu, Lhasa, Tibetan Terrier, Maltese Dog]",Maltese Dog
5,Siberian Husky/001.jpeg,"[American Staffordshire Terrier, Dhole, Cardigan, Dingo, Borzoi, Saluki, Norwegian Elkhound, Eskimo Dog, Malamute, Siberian Husky]",Siberian Husky
